# Sentiment 

In [3]:
from textblob import TextBlob
userText = "Shut up! You are an idiot"
tb = TextBlob(userText)
subjectivity = tb.subjectivity
polarity = round(tb.polarity, 2)
print(polarity)

-0.8


# 1 try

In [5]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain import LLMChain
import sys
sys.path.append("../src")
from setup import get_openai_model, enable_tracing

MODEL = get_openai_model()
enable_tracing("text_blob")


# define the output
response_schemas = [
    ResponseSchema(name="sentiment", description="a sentiment label based on the user text. It should be either Negative, Positive or Neutral"),
    ResponseSchema(name="reason", description="""
    If the sentiment is Negative then return the reason why the user shouldn't have said that.
    If the sentiment is Positive then return a compliment.
    For Neutral then return a instruct for a better message. 
    """),
    ResponseSchema(name="reply", description="the best and friendliest replacement to the given user text")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# prompt template
template = """You are good at detecting human emotion. All emotions you know are Negative, Positive and Neutral.
Given a human text, subjectivity and polarity, your job is to answer as best as possible.
Know that subjectivity is a measure of how much of a text is based on personal opinions or beliefs, rather than on facts. 
It is a float value between 0 and 1, where 0 represents an objective text and 1 represents a highly subjective text.
Also know that polarity is a indicator for the sentiment of the given user text, negative value means Negative, positive value means Positive and 0 means Neutral.
{format_instructions}
User text: {text}
Subjectivity: {subjectivity}
Polarity: {polarity}"""

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(template=template, input_variables=["text","subjectivity","polarity"],
                        partial_variables={"format_instructions": format_instructions})

# Build chain
sentiment_chain = LLMChain(llm=MODEL, prompt=prompt, output_key='result')

OPENAI ready
Enable tracing at text_blob


In [8]:
from textblob import TextBlob
userText = "Have you heard about the new school"
tb = TextBlob(userText)
subjectivity = tb.subjectivity
polarity = round(tb.polarity, 2)

ans = sentiment_chain({"text": userText, "polarity": polarity, "subjectivity": subjectivity})
ans

{'text': 'Have you heard about the new school',
 'polarity': 0.14,
 'subjectivity': 0.45454545454545453,
 'result': '285714285714285\n\n```json\n{\n\t"sentiment": "Neutral",\n\t"reason": "This statement is too vague and does not provide enough information.",\n\t"reply": "What do you know about the new school?"\n}\n```'}

# 2 try

In [21]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain import LLMChain
import sys
sys.path.append("../src")
from setup import get_openai_model, enable_tracing

MODEL = get_openai_model()
enable_tracing("text_blob")

emotions = ['Happy 😊','Sad 😔','Angry 😠','Surprise 😲','Fear 😨']

response_schemas = []
for emo in emotions:
    emos = emo.split(" ")
    des = f"""a js object contains two properties:
    label: str // always return '{emos[1]}' 
    score: int // a emotional score from 1 to 100 based the given user text
    reason: str // a reason for the score"""
    print(des)
    schema = ResponseSchema(name=emos[0], description=des)
    response_schemas.append(schema)
# define the output

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# prompt template
template = """You are good at detecting human emotion. All emotions you know are Negative, Positive and Neutral.
Given an user text, your job is to answer as best as possible.
{format_instructions}
User text: {text}"""

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(template=template, input_variables=["text"],
                        partial_variables={"format_instructions": format_instructions})

# Build chain
sentiment_chain = LLMChain(llm=MODEL, prompt=prompt, output_key='result')

OPENAI ready
Enable tracing at text_blob
a js object contains two properties:
    label: str // always return '😊' 
    score: int // a emotional score from 1 to 100 based the given user text
    reason: str // a reason for the score
a js object contains two properties:
    label: str // always return '😔' 
    score: int // a emotional score from 1 to 100 based the given user text
    reason: str // a reason for the score
a js object contains two properties:
    label: str // always return '😠' 
    score: int // a emotional score from 1 to 100 based the given user text
    reason: str // a reason for the score
a js object contains two properties:
    label: str // always return '😲' 
    score: int // a emotional score from 1 to 100 based the given user text
    reason: str // a reason for the score
a js object contains two properties:
    label: str // always return '😨' 
    score: int // a emotional score from 1 to 100 based the given user text
    reason: str // a reason for the score

In [22]:
ans = sentiment_chain({"text": "I'm feeling lonely and I need a hug so bad! Hu hu!"})
ans

{'text': "I'm feeling lonely and I need a hug so bad! Hu hu!",
 'result': '\n\n```json\n{\n\t"Happy": {\n        label: "😊",\n        score: 10,\n        reason: "The user expresses a need for companionship and comfort."\n    },\n\t"Sad": {\n        label: "😔",\n        score: 90,\n        reason: "The user expresses feelings of loneliness and sadness."\n    },\n\t"Angry": {\n        label: "😠",\n        score: 0,\n        reason: "No indication of anger in the user text."\n    },\n\t"Surprise": {\n        label: "😲",\n        score: 0,\n        reason: "No indication of surprise in the user text."\n    },\n\t"Fear": {\n        label: "😨",\n        score: 0,\n        reason: "No indication of fear in the user text."\n    }\n}\n```'}